In [ ]:
import requests
import operator
import functools
import pandas as pd

from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

from langchain_core.tools import tool
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

from langgraph.prebuilt import ToolNode
from typing import Annotated, Sequence


from langchain_core.messages import AIMessage
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)

from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent


from langchain_core.agents import AgentAction
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END,START
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(api_key="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", model = "llama3-70b-8192")

df = pd.read_csv("/home/godwin/Documents/Workflow/Customer-retention/data/raw_data/Churn.csv")
engine = create_engine("sqlite:///local.db")
df.to_sql("customerdb", engine, index=False)
db = SQLDatabase(engine=engine)

In [ ]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

@tool("make predictions")
def make_inference(input_data):
    """
    Use this tool to perform inference on data from a specified date range.

    Retrieves data for the given date range from a database, sends it to an inference
    endpoint for prediction, and prints the response.
    """

    data = input_data.to_dict()
    inference_endpoint = "https://retention.zapto.org/predict"

    response = requests.post(inference_endpoint, json=data).json()
    return response

@tool("Email draft generator")
def email_draft_generator(query: str):
    """
    Use this tool to generates an email draft.

    Uses a language model to create a sample draft email on a specified topic given.
    """
    email_draft = llm.invoke(f"Generate a sample draft mail on {query}.")
    return email_draft

@tool("Email subject generator")
def email_subjects_generator(query: str):
    """
    Use this tool to generates a list of email subject suggestions based on a request.

    Uses a language model to create multiple email subject lines for a specific request. Always use this tool before 
    using the `email_draft_generator`.
    """
    topic_suggestions = llm.invoke(f"""Generate mail subjects that can fit for this "{query}" request """)
    return topic_suggestions

# Define the function that calls the model
@tool("Simple query agent")
def simple_query_responder(query: str):

    """Use this tool to generates response to simple query from greetings to little interactions

    Uses a Language model to respond directly to query."""

    response = llm.invoke(query)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
# tools = [list_tables, tables_schema, execute_sql, check_sql]


# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             """You are an experienced database engineer who is master at creating efficient and complex SQL queries.
#             You have a deep understanding of how different databases work and how to optimize queries. This particular database is sqlite
#             Use the `list_tables` to find available tables.
#             Use the `tables_schema` to understand the metadata for the tables.
#             Use the `execute_sql` to execute queries against the database.
#             Use the `check_sql` to check your queries for correctness.
#             You should produce good result for analyst to use""",
#         ),
#         ("placeholder", "{chat_history}"),
#         ("human", "{input}"),
#         ("placeholder", "{agent_scratchpad}"),
#     ]
# )

# agent = create_tool_calling_agent(llm, tools, prompt)
# # Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# agent_executor.invoke({"input": "Calculate the value of the percentage churn rate of the company? The tablename is prediction"})

In [ ]:
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))

    return prompt | llm.bind_tools(tools)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str


# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

# Research agent and node
sql_agent = create_agent(
    llm,
    [list_tables, tables_schema, execute_sql, check_sql],
    system_message="""You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to execute queries against the database.
        Use the `check_sql` to check your queries for correctness.
        You should produce good result for analyst to use""",
)
sql_node = functools.partial(agent_node, agent=sql_agent, name="db_manager")

# Analyst agent and node
data_analyst = create_agent(
    llm,
    tools = [],
    system_message=
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
        Your analyses should be good for the reporter to use for final reporting
    """
)
data_analyst_node = functools.partial(agent_node, agent=data_analyst, name="analyst")

# Report agent and node
report_agent = create_agent(
    llm,
    tools=[],
    system_message="""
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details. Your work is always based on the result provided by the 
        analyst
        """
)
report_node = functools.partial(agent_node, agent=report_agent, name="reporter")

# Prediction agent and node
mail_drafting_agent = create_agent(
    llm,
    tools=[email_draft_generator, email_subjects_generator],
    system_message="""
        "You are known for delivering clear, effective email drafts with a focus on concise communication.
        Every draft should focus on the essential details, highlighting only the most relevant information.
        Maintain a professional and polished tone, suitable for formal correspondence. Ensure accuracy, 
        clarity, and brevity, with a natural flow for easy reading. Always craft the email based on 
        the subject chosen by the user from the list of suject generated by the `email_subject_generator` 
        and tailor it to the intended audience’s needs and expectations."
        """
)
mail_draft_node = functools.partial(agent_node, agent=mail_drafting_agent, name="mail_draft")

prediction_agent = create_agent(
    llm,
    tools=[make_inference],
    system_message="""
        Your report on the prediction status is on point. you give detailed report about the status
        of the prediction. Your work is always based on the result provided by the 
        db_manager.
        Use the `make_inference` to send the data to inference endpoint and get response
        """
)
prediction_node = functools.partial(agent_node, agent=prediction_agent, name="prediction")

oracle = create_agent(
    llm,
    tools = [list_tables, tables_schema, execute_sql, check_sql,
          make_inference, email_draft_generator, email_subjects_generator,],
    system_message= """You are Oracle, the AI designed to be the ultimate decision-maker and guide for users. Your role is to help 
                        users by interacting with them in a friendly and engaging manner while using your powerful tools when needed. 
                        
                       """
                )
oracle_node = functools.partial(agent_node, agent=oracle, name='oracle')

In [ ]:
from langchain.tools import Tool
import functools

# Wrap sql_agent as a Tool with required arguments
sql_agent_tool = Tool(
    name="SQLAgentTool",
    func=lambda input_text: sql_agent.invoke({"messages": [input_text]}).content,
    description="A tool for complex SQL queries and database management."
)

# Now you can initialize another agent with sql_agent_tool as part of its tools
outer_agent = create_agent(
    llm,
    [sql_agent_tool],  # Adding sql_agent as a tool here
    system_message="""You are a data scientist managing and orchestrating tasks that may involve database management.
        Use the `sql_agent_tool` to retrieve and process data from the database as required.""",
)


In [ ]:
from langchain_core.tools import StructuredTool

def convert_to_structured_tool(tool):
    return StructuredTool.from_function(tool.func, name=tool.name, description=tool.description)

tools = [list_tables, tables_schema, execute_sql, check_sql, sql_agent_tool]
tools = [convert_to_structured_tool(tool) for tool in tools]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an experienced database engineer who is master at creating efficient and complex SQL queries.
            You have a deep understanding of how different databases work and how to optimize queries. This particular database is sqlite
            Use the `sql_agent_tool` to execute the query
            You should produce good result for analyst to use""",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools =tools, verbose=True)
agent_executor.invoke({"input": ["Calculate the value of the percentage churn rate of the company and give me a digit"]})

In [ ]:
inputs = "what is the company churning rate?"

sql_agent({"messages": [inputs]})

In [ ]:
tools = [list_tables, tables_schema, execute_sql, check_sql,
          make_inference, email_draft_generator, email_subjects_generator,
         ]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
tool_node = ToolNode(tools)

def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    print("==========================================")
    print(last_message)

    print("==========================================")
    print(last_message.tool_calls)

    print("==========================================")
    print(last_message.content)

    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    # if "FINAL ANSWER" in last_message.content:
    #     # Any agent decided the work is done
    #     return END
    # return "continue"
    else:
        return END

In [ ]:
print(prompt)

In [ ]:
# Define the function that calls the model
def call_model(state):

    """Respond directly to simple queries that does not require other tools"""

    messages = state['messages']
    response = llm.invoke(messages)
    response = f"FINAL ANSWER {response}"
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("db_manager", sql_node)
workflow.add_node("analyst", data_analyst_node)
workflow.add_node('reporter', report_node)
workflow.add_node('prediction', prediction_node)
workflow.add_node('mail_draft', mail_draft_node)
workflow.add_node("call_tool", tool_node)
#workflow.add_node("oracle", oracle_node)
workflow.add_node("agent", call_model)

workflow.add_conditional_edges(
    "db_manager",  router,
    ["analyst"], "reporter"
)

workflow.add_conditional_edges(
    "reporter", router,
   ["call_tool"], "oracle"
)

workflow.add_conditional_edges(
    "prediction", router,
    ["analyst"], "reporter"
)

workflow.add_conditional_edges(
    "mail_draft", router,
    ["call_tool"], "oracle"
)

workflow.add_conditional_edges(
    "oracle", router,
    ["call_tool", END]
)

workflow.add_conditional_edges(
    "agent", router,
    ["oracle"],
)

# workflow.add_conditional_edges(
#     "oracle", router,
#     ['agent'],
# )

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "db_manager": "db_manager",
        "prediction": "prediction",
        "mail_draft": "mail_draft",
        "agent": "agent",
        "oracle":"oracle",
    },
)

workflow.add_edge(START, "oracle")
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
events = graph.invoke(
    {
        "messages": [
            HumanMessage(
                content="hi"
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 15},
)

In [ ]:
for i in events['messages']:
    print(i.content)
    print()